<a href="https://colab.research.google.com/github/jerry26432341/myTheses/blob/main/0609_%E6%A8%A1%E5%9E%8B%E5%BC%95%E5%85%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy
!python -m spacy download zh_core_web_sm
!wget -O chineseStopWords.txt https://raw.githubusercontent.com/tomlinNTUB/Machine-Learning-Basics/main/Chinese%20Stopwords.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 37.5 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2
2023-06-08 13:02:24.553750: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 13:02:25.775357: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 3

In [ ]:
import pandas as pd
import numpy as np
import jieba
import re
import gensim
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, vstack
from sklearn.svm import LinearSVC
from joblib import dump
from sklearn.model_selection import train_test_split
from google.colab import drive

# 掛載Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
print("在 cat 列中总共有 %d 个空值." % df['cat'].isnull().sum())
print("在 review 列中总共有 %d 个空值." % df['review'].isnull().sum())
df[df.isnull().values==True]
df = df[pd.notnull(df['review'])]

NameError: ignored

In [ ]:
df['cat_id'] = df['cat'].factorize()[0]
cat_id_df = df[['cat', 'cat_id']].drop_duplicates().sort_values('cat_id').reset_index(drop=True)
cat_to_id = dict(cat_id_df.values)
id_to_cat = dict(cat_id_df[['cat_id', 'cat']].values)
df.sample(10)

,cat,label,review,cat_id,filtered_review
13445,平板,0,大家不要买！大家不要买！大家不要买！大家不要买！大家不要买！大家不要买！大家不要买！大家不要...,1,买 买 买 买 买 买 买 买 买 买 买 买 买 买 买 买
54011,酒店,1,房间很干净。餐厅饭菜一般般。可惜没有独立的沙滩，需要步行一段路才能到海边。补充点评 2007...,9,房间 干净 餐厅 饭菜 般 可惜 独立 沙滩 步行 段 路 海边 补充 点评 2007年 1...
50756,计算机,1,目前3k以下的笔记本很少有货的，这个相对来说还算可以。配置比较全，带摄像头，整体做工尚可，装...,8,3k 笔记本 少有 货 相对来说 还算 配置 全 摄像头 整体 做工 尚 装ghost版 系...
32423,洗发水,0,东西用着跟以前买的不一样，洗发液很稀，而且不出泡沫。很失败的一次购物，一直都很信任京东， 这...,4,东西 买 洗 发液 稀 泡沫 失败 次 购物 信任 京东 洗发液 真心 专业 鉴定 信口开河...
53448,酒店,1,位置很好，就在南京路步行街后面，逛街购物很方便，然后离人民广场地铁站也近，交通很方便。,9,位置 南京 路步 行街 逛街 购物 广场 地铁站 交通
11930,平板,0,刚下单就降价，不舒服,1,下单 降价 舒服
58065,酒店,0,看中的是离机场近早上530起有专车送。但是如果是晚上到达的话太不安全了 周围什么都没有 的士...,9,看中 机场 早上 专车 送 晚上 到达 太 的士 黑车 白天 没事 地铁站 走 想 搭乘 早...
23981,水果,0,在这家买过好几次了，说实话，这是最后一次了。苹果越来越小，肯定没有10斤，也懒得计较了。为什...,3,这家 买 好几 次 说 实话 这是 次 苹果 越来越 肯定 斤 懒得 计较 生意 做 回头客...
51398,计算机,0,1发热厉害，用一个小时下面就烫手了，Dell的3000出头的笔记本一天也没这么热，有点害怕 ...,8,发热 厉害 小时 烫手 Dell 出头 笔记本 天 热 害怕 2机器 感觉 挺 单薄 塑料感...
28942,洗发水,1,非常好的商品，物流也快很好的商品,4,商品 物流 商品


In [ ]:
# 读取预处理的数据
df = pd.read_csv('preprocessed_data.csv')

In [ ]:
from joblib import load

# 载入TF-IDF向量化器
tfidf = load('/content/drive/MyDrive/tfidf_vectorizer.joblib')

d2v_model = gensim.models.Doc2Vec.load('/content/drive/MyDrive/doc2vec_model')

# 使用Doc2Vec模型对预处理后的文本进行向量化
d2v_vectors = []
for text in df['filtered_review']:
    if isinstance(text, str):
        vec = d2v_model.infer_vector(text.split())
        d2v_vectors.append(vec)
    else:
        d2v_vectors.append(np.zeros(d2v_model.vector_size))  # 对于非字符串值，填充为零向量
d2v_vectors = np.array(d2v_vectors)


# 使用hstack将TF-IDF向量和Doc2Vec向量连接起来
feature_matrix = hstack((tfidf_vectors, d2v_vectors))

# 定义训练数据
X_train = feature_matrix

# 定义标签
y_train = df['cat']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, df['cat'], test_size=0.2, random_state=42, shuffle=True)


# 载入训练好的LinearSVC模型
model = load('/content/drive/MyDrive/linear_svc_model.joblib')

# 进行预测
predictions = model.predict(X_test)

# 重置X_test的索引并创建一个新的DataFrame
X_test_df = pd.DataFrame(X_test.toarray())
X_test_df.reset_index(drop=True, inplace=True)

# 创建一个新的数据框存储预测结果
df_predicted = pd.DataFrame({'review': df['review'].values[X_test_df.index], 'predicted_category': predictions})

# 输出预测结果
print(df_predicted)

                                                  review predicted_category
0      ﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持...                 平板
1      作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...                洗发水
2      作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...                 平板
3      作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...                 平板
4      作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...                计算机
...                                                  ...                ...
12550                                 直接不能用，刚来点没反应，屏幕不能用                 酒店
12551            收到了，有点后悔，除了硬件升级之外，外观比x1丑太多了，无比丑的大黑边，崩溃中                 蒙牛
12552                                           一般，还没给赠品                 酒店
12553  京东描述带手机卡的分辨率是1920×1080，结果只有1280×800，多花这些钱不是为了能...                 衣服
12554                                   刚拿到手 内存一半就没了 太坑了                 蒙牛

[12555 rows x 2 columns]


In [ ]:
from google.colab import files
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# 进行预测
predictions = model.predict(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, predictions)

# 计算召回率
recall = recall_score(y_test, predictions, average='weighted')

# 计算精确率
precision = precision_score(y_test, predictions, average='weighted')

# 计算F1分数
f1 = f1_score(y_test, predictions, average='weighted')

# 创建包含预测结果的 DataFrame，仅使用测试数据集的索引
df_results = pd.DataFrame({'Review': df['review'].values[X_test_df.index], 'Predicted_Category': predictions, 'Actual_Category': y_test})

# 将预测结果分为预测正确和预测错误的两个数据框，仅使用测试数据集的索引
correct_predictions = df_results[df_results['Predicted_Category'] == df_results['Actual_Category']]
incorrect_predictions = df_results[df_results['Predicted_Category'] != df_results['Actual_Category']]

# 将预测正确和预测错误的结果保存为 Excel 文件
correct_predictions.to_excel('correct_predictions.xlsx', index=False)
incorrect_predictions.to_excel('incorrect_predictions.xlsx', index=False)

# 下载文件到本地
files.download('correct_predictions.xlsx')
files.download('incorrect_predictions.xlsx')

# 输出指标结果
print("准确率:", accuracy)
print("召回率:", recall)
print("精确率:", precision)
print("F1分数:", f1)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

准确率: 0.8615690959776982
召回率: 0.8615690959776982
精确率: 0.8658133236697655
F1分数: 0.862647410826731
